In [9]:
from dotenv import load_dotenv
import os
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.messages import HumanMessage
import requests

In [10]:
# Load environment variables from .env file
load_dotenv()

# Retrieve the API key
api_key = os.getenv("OPENAI_API_KEY")

In [11]:
@tool
def get_stock_price(ticker: str) -> float:
   """Get the stock price for the stock exchange ticker for the company."""
   api_url = f"https://finance.yahoo.com/quote/{ticker}"
   response = requests.get(api_url)
   if response.status_code == 200:
       data = response.json()
       return data["price"]
   else:
       raise ValueError(f"Failed to fetch stock price for {ticker}")
 

In [12]:
# Initialize the LLM with GPT-4o and bind the tools
llm = ChatOpenAI(model_name="gpt-4o", temperature=0, api_key=api_key)
llm_with_tools = llm.bind_tools([get_stock_price])

In [13]:
messages = [HumanMessage("What is the stock price of Apple?")]
 
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

In [ ]:
for tool_call in ai_msg.tool_calls:
   tool_msg = get_stock_price.invoke(tool_call)
  
   print(tool_msg.name)
   print(tool_call['args'])
   print(tool_msg.content)
   messages.append(tool_msg)
   print()
 
final_response = llm_with_tools.invoke(messages)
print(final_response.content)